# Day 18 - 2D cellular automation

Today's puzzle is touching upon [cellular automation](https://en.wikipedia.org/wiki/Cellular_automaton) again, but this time we have a finite 2D grid and 3 states, rather than just on or off.

Because this is a grid, I'll use numpy for this version. We can use [`scipy.signal.convolve2d()`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.signal.convolve2d.html) to give us neighbour counts, and [`numpy.select()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.select.html) to create the next step output grid.

Given a 3 x 3 array named `kernel` of 1s and a central 0 (indicating what surrounding cells to count),  `convolve2d()`, together with a boolean array will give us a count of surrounding `True` values for each cell. So if lumberyards are represented by the integer value `3`, then we can get a count of adjacent lumberyards with:

```python
>>> kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
>>> print(forest)  # custom class, ._matrix is the numpy array
.#|#
|||#
#|#|
...#
>>> convolve2d(forest._matrix == 3, kernel, mode='same')
array([[1, 0, 3, 1],
       [2, 3, 4, 2],
       [0, 2, 2, 3],
       [1, 2, 2, 1]])
```

We have three rules, each applying to one of the three states, and all 3 are simple counts of the surrounding area. So you can make the following assignments:

- *trees*: if *open* and *surrounding tree count* is three or more
- *lumberyard*: if *trees* and *surrounding lumberyard count* is three or more
- *open*: if *lumberyard* and *surrounding lumberyard count* is zero *or* *surrounding trees count* is zero

So for each cell, we need three neighbor counts for each of the types.

We can cast those rules as broadcasted numpy tests, then use `np.select()` with those tests to select either the replacement value or the original input cell:

```python
output = np.select(
    [trees_cond, lumberyard_cond, open_cond],
    [trees, lumberyards, opens],
    default=forest.matrix
)
```

In [1]:
from enum import Enum
from scipy.signal import convolve2d
import numpy as np

class Acre(Enum):
    open = 1, '.'
    trees = 2, '|'
    lumberyard = 3, '#'
    
    def __new__(cls, int, value):
        instance = object.__new__(cls)
        instance._value_ = value
        instance.int = int
        return instance

_kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
    
class Forest:
    def __init__(self, forestmap: str) -> None:
        self._matrix = np.array([
            Acre(c).int for line in forestmap.splitlines() for c in line
        ]).reshape((forestmap.index('\n'), -1))
    
    def __str__(self) -> str:
        mapping = {a.int: a.value for a in Acre}
        return '\n'.join([''.join(map(mapping.__getitem__, row)) for row in self._matrix])
    
    def step(self) -> None:
        f = self._matrix
        counts = {acre: convolve2d(f == acre.int, _kernel, mode='same') for acre in Acre}
        rules = {
            Acre.trees: (  # currently open, and has 3 or more trees as neighbours
                (f == Acre.open.int) & (counts[Acre.trees] >= 3)
            ),
            Acre.lumberyard: (  # currently trees, and has 3 or more lumberyards as neighbours
                (f == Acre.trees.int) & (counts[Acre.lumberyard] >= 3)
            ),
            Acre.open: (  # currently lumberyard, but either missing neighbouring lumberyards or trees
                (f == Acre.lumberyard.int) & (
                    (counts[Acre.lumberyard] == 0) | (counts[Acre.trees] == 0)
                )
            ),
        }
        self._matrix = np.select(
            list(rules.values()),
            [np.full(f.shape, a.int) for a in rules],
            default=f)
        
    def run(self, minutes: int) -> int:
        """Run for the given amount of time, then return the resource value"""
        for _ in range(minutes):
            self.step()
        counts = dict(zip(*np.unique(self._matrix, return_counts=True)))
        return counts[Acre.trees.int] * counts[Acre.lumberyard.int]

In [2]:
testforest = Forest('''\
.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.''')
assert testforest.run(10) == 1147

In [3]:
import aocd

data = aocd.get_data(day=18, year=2018)

In [4]:
print('Part 1:', Forest(data).run(10))

Part 1: 564375
